#### **Q3: BUBBLE POINT METHOD – MULTICOMPONENT MULTISTAGE DISTILLATION** 
1000 kmol/h of a saturated-liquid mixture of 60% methanol (normal boiling point 65 C), 20 mol% ethanol 
(normal boiling point 98 C), and 20 mol% n-propanol (normal boiling point 97 C) is fed to the middle 
stage of a distillation column having three equilibrium stages, a total condenser, a partial reboiler, all 
operated at 1 atm. The distillate rate is 600 kmol/h, and the external reflux rate is 2,000 kmol/h of 
saturated liquid.

In [ ]:
#### **Q3: BUBBLE POINT METHOD – MULTICOMPONENT MULTISTAGE DISTILLATION** 
""" 
1000 kmol/h of a saturated-liquid mixture of 60% methanol (normal boiling point 65 C), 20 mol% ethanol 
(normal boiling point 98 C), and 20 mol% n-propanol (normal boiling point 97 C) is fed to the middle 
stage of a distillation column having three equilibrium stages, a total condenser, a partial reboiler, all 
operated at 1 atm. The distillate rate is 600 kmol/h, and the external reflux rate is 2,000 kmol/h of 
saturated liquid.
"""
# Write your own code to obtain converged solutions of the temperature profiles.


# Algorithm for obtaining converged solutions of the temperature profiles.
# 1. Assume an initial temperature profile.
# 2. Calculate the bubble point temperature of each stage.
# 3. Calculate the dew point temperature of each stage.
# 4. Calculate the average temperature of each stage.
# 5. Update the temperature profile.
# 6. Repeat steps 2-5 until the temperature profile converges.


In [ ]:
import numpy as np    
def calculate_Pij(A :np.array,B :np.array,C :np.array,T_j :np.array) -> np.array:
        """
        Calculate the saturation pressure of a component at a given temperature.

        Parameters:
        A : float
            Antoine's coefficient A.
        B : float
            Antoine's coefficient B.
        C : float
            Antoine's coefficient C.
        T : float
            Temperature in Celsius.

        Returns:
        float
            Saturation pressure in atm.
        """
        return np.array([[10**(A[i] - B[i]/(T_j[j] + C[i])) for j in range(len(T_j))] for i in range(3)])


In [10]:
A= [7.89750, 7.28610, 7.01050]
B= [1474.08, 1251.33, 1152.21]
C= [229.664, 229.664, 229.664]
T_j= [65, 98, 97]

print(calculate_Pij(A,B,C,T_j))



[[ 785.09243236 2504.63922632 2426.46128911]
 [1095.1321941  2931.95583891 2854.08359123]
 [1259.64939087 3119.33795651 3042.97046498]]


In [12]:
def calculatate_Kij(Pij_sat: np.array)-> np.array:
    """
    Calculate the Ki values for the components.

    Parameters:
    Pi_sat : list
        List of saturation pressures for the components.

    Returns:
    list
        List of Ki values for the components.
    """
    return [[Pij_sat[i][j] for i in range(len(Pij_sat))] for j in range(len(Pij_sat[0]))]

print(calculatate_Kij(calculate_Pij(A,B,C,T_j)))

[[785.0924323551616, 1095.132194103674, 1259.6493908746313], [2504.63922632249, 2931.9558389086556, 3119.3379565135674], [2426.4612891144425, 2854.0835912281036, 3042.97046498474]]


In [13]:
def thomas_algorithm( A: np.array, B: np.array, C: np.array, D: np.array)-> np.array:
    """
    Solve a tridiagonal system of equations using the Thomas algorithm.

    Parameters:
    A : list
        Lower diagonal elements.
    B : list
        Main diagonal elements.
    C : list
        Upper diagonal elements.
    D : list
        Right-hand side vector.

    Returns:
    list
        Solution vector.
    """
    n = len(B)
    c_prime = np.zeros(n)
    d_prime = np.zeros(n)
    x = np.zeros(n)
    
    c_prime[0] = C[0]/B[0]
    d_prime[0] = D[0]/B[0]
    
    for i in range(1, n):
        c_prime[i] = C[i]/(B[i] - A[i]*c_prime[i-1])
        d_prime[i] = (D[i] - A[i]*d_prime[i-1])/(B[i] - A[i]*c_prime[i-1])
    
    x[-1] = d_prime[-1]
    
    for i in range(n-2, -1, -1):
        x[i] = d_prime[i] - c_prime[i]*x[i+1]
    
    return x

A =[0,100,100,200,200]
B =[-150,-344.5,-525.5,-605,-545]
C =[244.5,325.5,405,495,0]
D= [0,0,-30,0,0]

print(thomas_algorithm(A,B,C,D))

[0.56640823 0.34748971 0.19376154 0.09153758 0.03359177]


In [17]:
def check_summation( Xij: np.array) -> bool:
    """
    Check if the summation of Xij is equal to 1.

    Parameters:
    Xij : list
        List of component mole fractions.

    Returns:
    bool
        True if the summation is equal to 1, False otherwise.
    """
    return np.isclose(sum(Xij), 1, atol=1e-10)
print(check_summation([0.2, 0.3, 0.5]))

True


In [38]:
def calculate_Lj( feed_tray: int , Vj: np.array)-> np.array:
    """
    Calculate the liquid flow rates for each component.

    Parameters:
    Kij : list
        List of Ki values for the components.
    Vj : list
        List of vapor flow rates.

    Returns:
    list
        List of liquid flow rates for each component.
    """

    Lj = np.zeros_like(Vj)

    # Lj Before feed tray
    for i in range(feed_tray-1):
        Lj[i] = Vj[i+1] + 1
        
    # Lj After feed tray
    for i in range(feed_tray-1, 3-1):
        Lj[i] = Vj[i+1] + 2+ 1
        
    return Lj

In [39]:
calculate_Lj(2, [0, 0.3, 0.5])


array([1.3, 3.5, 0. ])

In [41]:
def calculate_Aj(Lj: np.array)-> np.array:
    """
    Calculate the Aj values for each component.

    Parameters:
    Lj : list
        List of liquid flow rates for each component.
    Vj : list
        List of vapor flow rates.

    Returns:
    list
        List of Aj values for each component.
    """
    # Aj = Lj-1
    Aj = np.zeros_like(Lj)
    Aj[0] = 0
    Aj[1:] = Lj[:-1]
    return Aj

In [42]:
calculate_Aj(calculate_Lj(2, [0, 0.3, 0.5]))

array([0. , 1.3, 3.5])

In [48]:
def calculate_Dj(feed_tray :int)-> np.array:
    """
    Calculate the Dj values for each component.

    Returns:
    list
        List of Dj values for each component.
    """
    # Dij = 0
    Dij = np.zeros((3, 3))
    Dij[feed_tray-1] = - 100 * np.array([0.6, 0.2, 0.2])
    return Dij

In [49]:
calculate_Dj(2)

array([[  0.,   0.,   0.],
       [-60., -20., -20.],
       [  0.,   0.,   0.]])

In [50]:
def thomas_algorithm(A: np.array, B: np.array, C: np.array, D: np.array)-> np.array:
    """
    Solve a tridiagonal system of equations using the Thomas algorithm.

    Parameters:
    A : list
        Lower diagonal elements.
    B : list
        Main diagonal elements.
    C : list
        Upper diagonal elements.
    D : list
        Right-hand side vector.

    Returns:
    list
        Solution vector.
    """
    n = len(B)
    c_prime = np.zeros(n)
    d_prime = np.zeros(n)
    x = np.zeros(n)
    
    c_prime[0] = C[0]/B[0]
    d_prime[0] = D[0]/B[0]
    
    for i in range(1, n):
        c_prime[i] = C[i]/(B[i] - A[i]*c_prime[i-1])
        d_prime[i] = (D[i] - A[i]*d_prime[i-1])/(B[i] - A[i]*c_prime[i-1])
    
    x[-1] = d_prime[-1]
    
    for i in range(n-2, -1, -1):
        x[i] = d_prime[i] - c_prime[i]*x[i+1]
    
    return x

In [51]:
A = [0, 100, 100, 200, 200]
B = [-150, -344.5, -525.5, -605, -545]
C = [244.5, 325.5, 405, 495, 0]
D = [0, 0, -30, 0, 0]

thomas_algorithm(A, B, C, D)

array([0.56640823, 0.34748971, 0.19376154, 0.09153758, 0.03359177])